# Training and testing target-specific machine-learning models for structure-based virtual screening

This Jupyter notebook helps users train and test target-specific classification-based machine-learning models and Regression-based machine learning models for structure-based virtual screening using PLEC and GRID features.

Additional information can be found in our Nature Protocols paper: Tran-Nguyen, V. K., Junaid, M., Simeon, S. & Ballester, P. J. A practical guide to machine-learning scoring for structure-based virtual screening. Nat. Protoc. (2023)

We recommend users to set up the protocol-env environment before running the code in this Jupyter notebook. This can be done using the protocol-env.yml file in our MLSF-protocol github repository: https://github.com/vktrannguyen/MLSF-protocol.

For deepcoys generation, please use the github repository https://github.com/fimrie/DeepCoy/tree/master

In [ ]:
import os
import numpy as np
import pandas as pd
import oddt
from oddt.fingerprints import PLEC
from scipy import stats
from sklearn import preprocessing
import pickle
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from xgboost.sklearn import XGBClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from deepchem.utils import download_url, load_from_disk
from deepchem.utils.vina_utils import prepare_inputs
from deepchem.models import AtomicConvModel
from deepchem.feat import RdkitGridFeaturizer
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval

## 2. Load smile files for train and test

In [ ]:
#Provide the pathway to the csv training data file:
train_data = pd.read_csv("pathway_to_training-set_csv_data_file")

#Provide the pathway to the csv test data file:
test_data = pd.read_csv("pathway_to_test-set_csv_data_file")

## 3. Convert smiles to mol2 files

In [ ]:
for index,smi in enumerate(chem_file_1_smiles):
    mol=pybel.readstring(string=smi,format='smiles')
    mol.title='mol_'+str(index)
    mol.make3D('mmff94s')
    mol.localopt(forcefield='mmff94s', steps=500)
    out=pybel.Outputfile(filename='/path_for_mol2_files/'+'mol_'+str(index)+'.mol2',format='mol2',overwrite=True)
    out.write(mol)
    out.close()

## 4. Run docking using smina

In [ ]:
def run_smina(input_file):
    receptor_path = ['path_to_receptor_pdb_file']
    input_path = ['path_to_mol2_files' + str(input_file)]
    output_path = ['path_for_output_docked_poses/' + str(input_file)[:-5] +  '_docked.sdf']
    smina_command = ['path_to_smina_code/smina -r '+ str(receptor_path[0])+ 
                    ' -l '+str(input_file)+ ' -o '+str(output_path[0])+ ' --center_x ' +str(center['center_x'])+
                    ' --center_y '+str(center['center_y'])+ ' --center_z '+str(center['center_z'])+ 
                    ' --size_x '+ str(size['size_x'])+ ' --size_y '+ str(size['size_y'])+ ' --size_z '+ str(size['size_z'])+ ' --exhaustiveness 8 --num_modes 1']
    os.system(smina_command[0])
    
Parallel(n_jobs = 40, backend = 'multiprocessing')(delayed(run_smina)(input_file) for input_file in tqdm(input_files))

## 5. Extract PLEC fingerprints from input data

In [ ]:
protein = next(oddt.toolkit.readfile('pdb', 'path_to_receptor_structure'))
def parallel_plec(lig):
    ligand = next(oddt.toolkit.readfile('sdf', lig))
    feature = PLEC(ligand, protein = protein, size = 4092, 
                  depth_protein = 4, depth_ligand = 2,
                  distance_cutoff = 4.5, sparse = False)
    return feature
plec_training_actives = Parallel(n_jobs = 40, backend = "multiprocessing")(delayed(parallel_plec)(mol) for mol in tqdm(docked_sdf_active))

## 6. Extract GRID features from input data

In [ ]:
featurizer = RdkitGridFeaturizer(voxel_width=16.0, feature_types=["ecfp", "splif", "hbond", "salt_bridge"], ecfp_power=9,splif_power=9,flatten=True)
protein = 'path_to_receptor_structure'
def extract_grid_feature(ligand_file):
    feature = featurizer._featurize((ligand_file, protein))
    return feature
grid_training_augmented_actives = Parallel(n_jobs = 40, backend = "multiprocessing")(delayed(extract_grid_feature)(mol) for mol in tqdm(docked_sdf_active))

## 7. Perform cross validation

In [55]:
def find_optimal_threshold(y_true, y_pred_prob):
    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    optimal_idx = np.argmax(tpr - fpr)
    return thresholds[optimal_idx]

# Initialize the StratifiedKFold object
skf = StratifiedKFold(n_splits=5)

# Define the models in a dictionary
models = {
    'Random Forest': RandomForestClassifier(max_depth=3, max_features='log2', min_samples_leaf=1, min_samples_split=8, n_estimators=270, n_jobs=40),
    'XGBoost': XGBClassifier(learning_rate=0.01, max_depth=7, colsample_bytree=0.73, gamma=1.96, min_child_weight=8, subsample=0.71, n_estimators=150),
    'ANN': MLPClassifier(hidden_layer_sizes=(50,), activation='tanh', alpha=0.0070, learning_rate='invscaling', solver='sgd')
}

# Perform stratified 5-fold cross-validation for each model
results = {}

for model_name, model in models.items():
    print(f"Evaluating {model_name}...")
    
    metrics = {
        'Optimal Threshold': [],
        'Average Precision': [],
        'ROC-AUC': [],
        'PR-AUC': [],
        'Precision': [],
        'Recall': [],
        'MCC': [],
        'F1 Score': []
    }
    
    fold = 1
    for train_index, test_index in skf.split(X_plec_train, y_plec_train):
        # Split the data into training and validation sets
        X_train, X_test = X_plec_train.iloc[train_index], X_plec_train.iloc[test_index]
        y_train, y_test = y_plec_train[train_index], y_plec_train[test_index]
        
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict probabilities on the validation set
        y_pred_prob = model.predict_proba(X_test)[:, 1]
        
        # Find optimal threshold
        optimal_threshold = find_optimal_threshold(y_test, y_pred_prob)
        
        # Make predictions using the optimal threshold
        y_pred = (y_pred_prob >= optimal_threshold).astype(int)
        
        # Calculate metrics
        avg_precision = average_precision_score(y_test, y_pred_prob)
        roc_auc = roc_auc_score(y_test, y_pred_prob)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        mcc = matthews_corrcoef(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        # Calculate PR-AUC
        precision_curve, recall_curve, _ = precision_recall_curve(y_test, y_pred_prob)
        pr_auc = auc(recall_curve, precision_curve)
        
        # Store metrics
        metrics['Optimal Threshold'].append(optimal_threshold)
        metrics['Average Precision'].append(avg_precision)
        metrics['ROC-AUC'].append(roc_auc)
        metrics['PR-AUC'].append(pr_auc)
        metrics['Precision'].append(precision)
        metrics['Recall'].append(recall)
        metrics['MCC'].append(mcc)
        metrics['F1 Score'].append(f1)
        
        print(f"Fold {fold} - Optimal Threshold: {optimal_threshold:.3f}, Avg Precision: {avg_precision:.3f}, ROC-AUC: {roc_auc:.3f}, PR-AUC: {pr_auc:.3f}, Precision: {precision:.3f}, Recall: {recall:.3f}, MCC: {mcc:.3f}, F1: {f1:.3f}")
        fold += 1
    
    # Calculate mean and standard deviation of each metric
    results[model_name] = {metric: (np.mean(values), np.std(values)) for metric, values in metrics.items()}

# Print overall results
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric, (mean, std) in metrics.items():
        print(f"Mean {metric}: {mean:.3f}, Std Dev: {std:.3f}")

# Print mean metrics for each model
print("\nMean Metrics for Each Model (rounded to three decimal places):")
mean_metrics = {metric: [] for metric in metrics.keys()}
for model_name, metrics in results.items():
    print(f"\n{model_name}:")
    for metric, (mean, std) in metrics.items():
        mean_rounded = round(mean, 3)
        mean_metrics[metric].append(mean_rounded)
        print(f"Mean {metric}: {mean_rounded}, Std Dev: {std:.3f}")

print("\nMean Metric Scores for Each Model:")
for metric, scores in mean_metrics.items():
    print(f"{metric}: {scores}")

Evaluating Random Forest...
Fold 1 - Optimal Threshold: 0.024, Avg Precision: 0.872, ROC-AUC: 0.990, PR-AUC: 0.871, Precision: 0.330, Recall: 0.944, MCC: 0.543, F1: 0.489
Fold 2 - Optimal Threshold: 0.025, Avg Precision: 0.919, ROC-AUC: 0.995, PR-AUC: 0.919, Precision: 0.433, Recall: 0.963, MCC: 0.635, F1: 0.598
Fold 3 - Optimal Threshold: 0.026, Avg Precision: 0.829, ROC-AUC: 0.977, PR-AUC: 0.829, Precision: 0.411, Recall: 0.880, MCC: 0.589, F1: 0.560
Fold 4 - Optimal Threshold: 0.026, Avg Precision: 0.833, ROC-AUC: 0.975, PR-AUC: 0.832, Precision: 0.254, Recall: 0.935, MCC: 0.468, F1: 0.399
Fold 5 - Optimal Threshold: 0.028, Avg Precision: 0.803, ROC-AUC: 0.957, PR-AUC: 0.803, Precision: 0.247, Recall: 0.916, MCC: 0.456, F1: 0.389
Evaluating XGBoost...
Fold 1 - Optimal Threshold: 0.138, Avg Precision: 0.932, ROC-AUC: 0.992, PR-AUC: 0.932, Precision: 0.457, Recall: 0.972, MCC: 0.656, F1: 0.621
Fold 2 - Optimal Threshold: 0.183, Avg Precision: 0.983, ROC-AUC: 0.999, PR-AUC: 0.983, Prec

## 8. Train and test the RF algorithm

hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"n_estimators": hp.uniform("n_estimators", 50, 10000),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5, None]),
         "criterion": hp.choice("criterion", ['gini', 'entropy']),
         'min_samples_leaf':hp.randint('min_samples_leaf',1,5),
         'min_samples_split':hp.randint('min_samples_split',2,6)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(**space, n_jobs = 40)
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(fn = hyperparameter_tuning_randomforest, space = space, algo = tpe.suggest,
                              max_evals = 10, trials = trials)
best_params = space_eval(space, best_rf_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the RF model on the training molecules, using optimal parameters:
rf_plec = RandomForestClassifier(n_estimators = int(best_params['n_estimators']), 
                                 max_depth = best_params['max_depth'], 
                                 criterion = best_params['criterion'],
                                 min_samples_split = best_params['min_samples_split'],
                                 min_samples_leaf = best_params['min_samples_leaf']
                                 n_jobs = 30)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_plec_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})
plec_result_rf.to_csv("where_you_want_to_store_the_csv_output_file")

## 9. Train and test the XGB algorithm

hyperparameter tuning

In [ ]:
seed=5
def objective_plec_2(params):
    xgb_plec=xgb.XGBClassifier(**params, n_jobs=40, random_state=seed)
    xgb_plec.fit(X_train_plec,y_train_plec)
    pred_xgb_plec=xgb_plec.predict(X_val_plec)
    accuracy = accuracy_score(y_val_plec, pred_xgb_plec)
    return {'loss': -accuracy, 'status': STATUS_OK}

params={'n_estimators':hp.randint('n_estimators',100,500),
           'max_depth':hp.randint('max_depth',5,20),
           'learning_rate':hp.choice('learning_rate',[0.01,0.1])}

def optimize_plec_2(trial_xgb_plec):

    best_plec_2=fmin(fn=objective_plec_2,
                     space=params,
                     algo=tpe.suggest,
                     max_evals=500,
                     rstate=np.random.default_rng(seed))
    return best_plec_2

trial_xgb_plec=Trials()
best_xgb_plec=optimize_plec_2(trial_xgb_plec)

In [ ]:
#Train the XGB model on the training molecules, using optimal parameters:
xgb_plec = XGBClassifier(max_depth = int(best_params['max_depth']),
                         n_estimators = int(best_params['n_estimators']),
                         learning_rate = int(best_params['learning_rate'])
                         n_jobs = 40, random_state = 0)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})
plec_result_xgb.to_csv("where_you_want_to_store_the_csv_output_file")

## 10. Train and test the ANN algorithm

hyperparameter tuning

In [ ]:
#Define the search space for optimal parameters:
space = {"hidden_layer_sizes": hp.uniform("hidden_layer_sizes", 8, 140),
         "activation": hp.choice("activation", ['relu', 'tanh']),
         "max_iter": hp.uniform("max_iter", 1000, 10000)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_ANN(space):
    model = MLPClassifier(hidden_layer_sizes = int(space['hidden_layer_sizes']),
                          activation = space['activation'],
                          max_iter = int(space['max_iter']))
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_ann_classification = fmin(fn = hyperparameter_tuning_ANN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_ann_classification)

#Optimal parameters:
best_params

In [ ]:
#Train the ANN model on the training molecules, using optimal parameters:
ann_plec = MLPClassifier(hidden_layer_sizes = int(best_params['hidden_layer_sizes']), 
                         activation = best_params['activation'], 
                         max_iter = int(best_params['max_iter']), 
                         random_state = 0)
ann_plec.fit(train_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})
plec_result_ann.to_csv("where_you_want_to_store_the_csv_output_file")